In [1]:
//common functions

function normalizeTime(loop) {
    let touch_pts = loop.slice(1, -1);
    const start_time = touch_pts[0].ts;
    const dur = touch_pts.slice(-1)[0].ts - start_time;
    touch_pts = touch_pts.map(({ts, pos}) => ({pos, ts: (ts-start_time)/dur}));
    return touch_pts;
}

function oscArgs(...args) {
    let output  = args.map(e => {
        let type_str = typeof e;
        switch(type_str) {
            case 'string':
                return {type: 's', value: e}
            case 'number': 
                return {type: 'f', value: e}
            case 'boolean':
                return {type: 'i', value: e+0}
        }
    });
    return output;
}

In [2]:
//TCP server to OF

const net = require('net');
const tcp_port = 11999;
const tcp_host = '127.0.0.1';

const server = net.createServer();
server.listen(tcp_port, tcp_host, () => {
    console.log('TCP Server is running on port ' + tcp_port + '.');
});

let sockets = [];

server.on('connection', function(sock) {
    console.log('CONNECTED: ' + sock.remoteAddress + ':' + sock.remotePort);
    sockets.push(sock);

    sock.write( JSON.stringify(processedLoops) + '\n' );

    sock.on('data', function(data) {
        console.log('DATA ' + sock.remoteAddress + ': ' + data);
        // Write the data back to all the connected, the client will receive it as data from the server
        sockets.forEach(function(sock, index, array) {
            sock.write(sock.remoteAddress + ':' + sock.remotePort + " said " + data + '\n');
        });
    });

    // Add a 'close' event handler to this instance of socket
    sock.on('close', function(data) {
        let index = sockets.findIndex(function(o) {
            return o.remoteAddress === sock.remoteAddress && o.remotePort === sock.remotePort;
        })
        if (index !== -1) sockets.splice(index, 1);
        console.log('CLOSED: ' + sock.remoteAddress + ' ' + sock.remotePort);
    });
});

Server {
  _events: [Object: null prototype] {
    listening: [Function: bound onceWrapper] { listener: [Function (anonymous)] },
    connection: [Function (anonymous)]
  },
  _eventsCount: 2,
  _maxListeners: undefined,
  _connections: 0,
  _handle: null,
  _usingWorkers: false,
  _workers: [],
  _unref: false,
  allowHalfOpen: false,
  pauseOnConnect: false,
  [Symbol(kCapture)]: false,
  [Symbol(async_id_symbol)]: -1
}

TCP Server is running on port 11999.


In [3]:
//OSC Server to touchOSC

const osc = require('osc');

const TCP_OSC_PORT = 7071;
const TCP_OSC_HOST = '192.168.1.39';

let tcp_osc_port = new osc.TCPSocketPort({});
// change to remote host/port (address of phone/tablet) - set touchOSC on device to TCPserver
tcp_osc_port.open(TCP_OSC_HOST, TCP_OSC_PORT); 

tcp_osc_port.on('ready', () => {
  console.log('ready TCP');
});


const UDP_OSC_PORT = 7071;
const UDP_OSC_HOST = '0.0.0.0';

// Bind to a UDP socket to listen for incoming OSC events.
let udp_osc_port = new osc.UDPPort({
    localAddress: UDP_OSC_HOST,
    localPort: UDP_OSC_PORT,
    remoteAddress: '127.0.0.1',
    remotePort: 7072
});

udp_osc_port.on("ready", () => {
    console.log('ready UDP');
});

udp_osc_port.open();

let osc_port = udp_osc_port;

const rawLoops = {};
const processedLoops = {};
let activeLoop = 'firstLoop';
let isRecording = false;
let isTouching = false;
const handlers = {};

function processLoop(loopKey) {
    processedLoops[loopKey] = rawLoops[loopKey];
}

handlers['/recording'] = (isRec) => {
    isRecording = isRec;
    if(isRecording) {
        rawLoops[activeLoop] = [{ts: Date.now(), pos: 'start'}];
    } else {
        rawLoops[activeLoop].push({ts: Date.now(), pos: 'end'});
        processLoop(activeLoop);
        sockets.forEach(sock => sock.write( JSON.stringify(processedLoops) + '\n' ))
    }
}
const f = float => ({type: 'f', value: float}); //helper for formatting floats for OSC
handlers['/pos'] = (x, y) => {
    if(isRecording) {
        rawLoops[activeLoop].push({ts: Date.now(), pos: {x, y}});
    }
    udp_osc_port.send({address: '/touchPos', args: [f(x), f(y)]});
}
handlers['/pos/touch'] = (touching) => {
    isTouching = touching;
    udp_osc_port.send({address: '/touching', args: [f(isTouching)]});
}

osc_port.on('message', (message) => {
    let {address, args} = message;
    console.log(message)
    if(handlers[address]) handlers[address](...args); 
});

EventEmitter {
  options: {
    localAddress: '0.0.0.0',
    localPort: 7071,
    remoteAddress: '127.0.0.1',
    remotePort: 7072
  },
  _events: [Object: null prototype] {
    data: [Function: bound ],
    open: [Function: bound ],
    ready: [Function (anonymous)],
    message: [Function (anonymous)]
  },
  _eventsCount: 4,
  socket: Socket {
    _events: [Object: null prototype] {
      error: [Array],
      listening: [Function: onListening]
    },
    _eventsCount: 2,
    _maxListeners: undefined,
    type: 'udp4',
    [Symbol(kCapture)]: false,
    [Symbol(async_id_symbol)]: 28,
    [Symbol(state symbol)]: {
      handle: [UDP],
      receiving: false,
      bindState: 1,
      connectState: 0,
      queue: undefined,
      reuseAddr: undefined,
      ipv6Only: undefined,
      recvBufferSize: undefined,
      sendBufferSize: undefined
    }
  }
}

ready UDP
{ address: '/recording', args: [ 1 ] }
{ address: '/pos/touch', args: [ true ] }
{ address: '/pos', args: [ 0.44062161445617676, 0.605151355266571 ] }
{ address: '/pos', args: [ 0.44062161445617676, 0.5996589660644531 ] }
{ address: '/pos', args: [ 0.43890997767448425, 0.5996589660644531 ] }
{ address: '/pos', args: [ 0.43890997767448425, 0.5992557406425476 ] }
{ address: '/pos', args: [ 0.4278794229030609, 0.5992557406425476 ] }
{ address: '/pos', args: [ 0.4278794229030609, 0.5919603109359741 ] }
{ address: '/pos', args: [ 0.41048917174339294, 0.5919603109359741 ] }
{ address: '/pos', args: [ 0.41048917174339294, 0.577514111995697 ] }
{ address: '/pos', args: [ 0.39066460728645325, 0.577514111995697 ] }
{ address: '/pos', args: [ 0.39066460728645325, 0.5580166578292847 ] }
{ address: '/pos', args: [ 0.35265102982521057, 0.5580166578292847 ] }
{ address: '/pos', args: [ 0.35265102982521057, 0.5184435844421387 ] }
{ address: '/pos', args: [ 0.31068167090415955, 0.518443584442

In [4]:
processedLoops['firstLoop'] = normalizeTime(processedLoops['firstLoop'])

[
  { pos: { x: 0.44062161445617676, y: 0.605151355266571 }, ts: 0 },
  {
    pos: { x: 0.44062161445617676, y: 0.5996589660644531 },
    ts: 0.0004677268475210477
  },
  {
    pos: { x: 0.43890997767448425, y: 0.5996589660644531 },
    ts: 0.03157156220767072
  },
  {
    pos: { x: 0.43890997767448425, y: 0.5992557406425476 },
    ts: 0.031805425631431246
  },
  {
    pos: { x: 0.4278794229030609, y: 0.5992557406425476 },
    ts: 0.03554724041159962
  },
  {
    pos: { x: 0.4278794229030609, y: 0.5919603109359741 },
    ts: 0.03578110383536015
  },
  {
    pos: { x: 0.41048917174339294, y: 0.5919603109359741 },
    ts: 0.03952291861552853
  },
  {
    pos: { x: 0.41048917174339294, y: 0.577514111995697 },
    ts: 0.03952291861552853
  },
  {
    pos: { x: 0.39066460728645325, y: 0.577514111995697 },
    ts: 0.04326473339569691
  },
  {
    pos: { x: 0.39066460728645325, y: 0.5580166578292847 },
    ts: 0.04349859681945744
  },
  {
    pos: { x: 0.35265102982521057, y: 0.55801665782928

CONNECTED: 127.0.0.1:53659


In [ ]:
sockets.forEach(sock => sock.write( JSON.stringify(processedLoops) + '\n' ))

In [9]:
udp_osc_port.send({address: "/launch", args: oscArgs('firstLoop', 2, false, 'grp1', 'key1')});

CLOSED: 127.0.0.1 59986


In [ ]:
processedLoops['firstLoop'][10]